# AlgorithmConfig API
首先创建 AlgorithmConfig 的一个实例，然后调用其某些方法来设置各种配置选项。RLlib 在其所有代码部分使用以下符合 black 标准的格式。

In [1]:
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig

config = (
    # Create an `AlgorithmConfig` instance.
    AlgorithmConfig()
    # Change the learning rate.
    .training(lr=0.0005)
    # Change the number of Learner actors.
    .learners(num_learners=2)
)
config.environment(env="CartPole-v1")  # call the proper method

## 算法特定的配置类
实践中不会直接使用基础 AlgorithmConfig 类，而总是使用其算法特定的子类，例如 PPOConfig。每个子类都有自己的一组 additional arguments 用于 training() 方法。


In [ ]:
from ray.rllib.algorithms.impala import IMPALAConfig

config = (
    # Create an `IMPALAConfig` instance.
    IMPALAConfig()
    # Specify the RL environment.
    .environment("CartPole-v1")
    # Change the learning rate.
    .training(lr=0.0004)
)

# Change an IMPALA-specific setting (the entropy coefficient).
config.training(entropy_coeff=0.01)

# Build the algorithm instance.
impala = config.build_algo()

# Further alter the config without affecting the previously built IMPALA object ...
config.training(lr=0.00123)
# ... and build a new IMPALA from it.
another_impala = config.build_algo()


2025-08-06 17:06:47,400	WARNING algorithm_config.py:5033 -- You are running IMPALA on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
/home/robotarm/anaconda3/envs/ray_env/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:520: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/robotarm/anaconda3/envs/ray_env/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You c

(SingleAgentEnvRunner pid=3957377) 2025-08-06 17:10:56,354	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
(raylet) [2025-08-06 17:11:48,775 E 3945255 3945255] (raylet) node_manager.cc:3041: 18 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 61fd26e6f3808fe0e771dcc37032101039e669fcb13ae215856e2c04, IP: 198.18.0.1) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 198.18.0.1`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs p

In [2]:
from ray import tune

tuner = tune.Tuner(
    "IMPALA",
    param_space=config,  # <- your RLlib AlgorithmConfig object
    run_config=tune.RunConfig(stop={"num_env_steps_sampled_lifetime": 4000}),
)
# Run the experiment with Ray Tune.
results = tuner.fit()

2025-08-06 17:20:54,176	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/robotarm/ray_results/IMPALA_2025-08-06_17-20-14' in 0.0252s.
2025-08-06 17:20:54,214	INFO tune.py:1041 -- Total run time: 39.70 seconds (39.60 seconds for the tuning loop).


## 通用配置设置
大多数配置设置是通用的，适用于 RLlib 的所有 Algorithm 类。以下部分将引导您了解用户在深入研究其他配置设置和开始超参数微调之前应密切关注的最重要的配置设置。
### RL 环境
config.environment("Humanoid-v5")
### 学习率 lr
config.training(lr=0.0001)
### 训练批次大小
config.training(train_batch_size_per_learner=256)
### 折扣因子 gamma
config.training(gamma=0.995)
### 使用 num_env_runners 和 num_learners 进行扩展
config.env_runners(num_env_runners=4)

'''Also use `num_envs_per_env_runner` to vectorize your environment on each EnvRunner actor.
Note that this option is only available in single-agent setups.
The Ray Team is working on a solution for this restriction. '''

config.env_runners(num_envs_per_env_runner=10)

config.learners(num_learners=2)
### 禁用 explore 行为
'''Disable exploration behavior.
When False, the EnvRunner calls `forward_inference()` on the RLModule to compute
actions instead of `forward_exploration()`. '''

config.env_runners(explore=False)
### Rollout 长度
config.env_runners(rollout_fragment_length=50)